In [1]:
import os
import gc
import shutil
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
import math

import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim

from imutils.video import FPS
import imutils

In [3]:
# cap = cv2.VideoCapture(-1)

# if not cap.isOpened():
#     print('fail')
#     exit()

In [4]:
test_transform = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
  # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
import cv2
import numpy as np
import torch
from imutils.video import FPS
import imutils
from PIL import Image

def mask_detector(frame, network, model):
    (h, w) = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    network.setInput(blob)
    detections = network.forward()

    faces = []
    locations = []
    predictions = []
    confidences = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence < 0.3:
            continue

        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype(int)

        startX = max(0, startX)
        startY = max(0, startY)
        endX = min(w - 1, endX)
        endY = min(h - 1, endY)

        if endX > startX and endY > startY:
            face = frame[startY:endY, startX:endX]
            
            try:
                face = Image.fromarray(face)
            except ValueError:
                continue

            face = test_transform(face)
            face = face.unsqueeze(0).to(device)

            faces.append(face)
            locations.append((startX, startY, endX, endY))
            confidences.append(confidence)

    if len(faces) > 0:
        with torch.no_grad():
            predictions = model(torch.cat(faces, dim=0))

    return (locations, predictions, confidences)

# 모델
face_model_path = './res10_300x300_ssd_iter_140000_fp16.caffemodel'
prototxt_config_path = './deploy.prototxt'
mask_detector_model_path = './best_model.pth' # mask_detector_1.pth

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # CUDA 사용

# input_shape = (3, 224, 224)
# model = CustomNet(input_shape=input_shape)

num_classes = 2

model = models.mobilenet_v2(pretrained = True)

# model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model.classifier[1] = nn.Sequential(
                      nn.Linear(1280, 256),
                      nn.ReLU(),
                      nn.Linear(256, 128),
                      nn.ReLU(),
                      nn.Dropout(0.4),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 32),
                      nn.ReLU(),
                      nn.Dropout(0.4),
                      nn.Linear(32, num_classes),
                      nn.Softmax(dim=1))

model.load_state_dict(torch.load(mask_detector_model_path, map_location=device))
model.eval()

network = cv2.dnn.readNet(prototxt_config_path, face_model_path)
model.to(device)

# # 비디오 설정
# video_path = './input.mov'
# vs = cv2.VideoCapture(video_path)

# # 웹캠 설정
vs = cv2.VideoCapture(0)

# 비디오 저장 설정
videoFileName = 'output.avi'
ret, frame = vs.read()

w = int(vs.get(cv2.CAP_PROP_FRAME_WIDTH))  # width
h = int(vs.get(cv2.CAP_PROP_FRAME_HEIGHT))  # height

fps = vs.get(cv2.CAP_PROP_FPS)  # frame per second
# fps = 25.0

# avc1, mp4v, MJPG
# *는 문자를 풀어쓰는 방식, *'DIVX' == 'D', 'I', 'V', 'X'
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter(videoFileName, fourcc, fps, (w, h), True)

# fps = FPS().start()

# writer = None

if vs.isOpened():
    while True:
        ret, frame = vs.read()
        
        if not ret:
            break
        
        # 좌우 반전
        if ret:
            frame = cv2.flip(frame, 1)
            
        frame = imutils.resize(frame, width=500)
        
        if frame is None:
            break

        (locations, predicts, confidences) = mask_detector(frame, network, model)

        for (box, predict, confidence) in zip(locations, predicts, confidences):
            (startX, startY, endX, endY) = box
            (without_mask, mask) = predict
            confidence_label = confidence

            label = "No Mask" if mask > without_mask else "Mask"

            if label == "Mask" and max(mask, without_mask) * 100 >= 70:
                color = (0, 255, 0)  # 초록
            elif label == "No Mask" and max(mask, without_mask) * 100 >= 70:
                color = (0, 0, 255)  # 빨강
            else:
                color = (0, 0, 255)  # 부적절한 마스크도 미착용(No Mask)에 포함

            label = "{}: {:.2f}%".format(label, max(mask, without_mask) * 100)

            cv2.putText(frame, label, (startX, startY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 2, cv2.LINE_AA)
            # cv2.putText(frame, f'Confidence: {confidence_label:.2f}', (startX, startY - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 2, cv2.LINE_AA)
            cv2.rectangle(frame, (startX, startY), (endX, endY), thickness=2, color=color)
            
        # ESC키 누르면 중지
        if cv2.waitKey(1) & 0xFF == 27:
            break
        
        # fps.update()
        
        # if writer is None:
        #     # *는 문자를 풀어쓰는 방식, *'DIVX' == 'D', 'I', 'V', 'X'
        #     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        #     writer = cv2.VideoWriter(videoFileName, fourcc, fps, (w, h), True)

        #비디오 저장
        if writer is not None:
            writer.write(frame)
            
        cv2.imshow("Mask Detector", frame)
        
else:
    print('not opened')

# # # fps 정지 및 정보 출력
# fps.stop()
# print("[재생 시간 : {:.2f}초]".format(fps.elapsed()))
# print("[FPS : {:.2f}]".format(fps.fps()))

# 종료
writer.release()
vs.release()
cv2.destroyAllWindows()
exit()

/Users/gimminsu/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gimminsu/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
2024-10-11 17:25:16.953 python[30001:1187717] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-11 17:25:16.953 python[30001:1187717] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [6]:
cv2.__version__
# 4.10.0.84

'4.10.0'